In [18]:
import time
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [19]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [20]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/nick/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [41]:
url = 'http://www.dpdconnect.com/dailybulletin.aspx'
browser.visit(url)

In [42]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#mainTable = soup.find('div', id='gbox_tblDB')
#listTable= soup.find('div', id="tblDB")
dateField = browser.find_by_id("mainContent_txtDate2")
dateSubmit = browser.find_by_id("mainContent_lbUpdate")
limit_selector = browser.find_by_css("#pager_center select")
type_selector = browser.find_by_id("mainContent_ddlType2")
print(type_selector.value)



AL


In [43]:
def nap(cat=.150):
    time.sleep(cat)


In [44]:
def rebaseHtml():
    nap()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mainTable = soup.find('div', id='gbox_tblDB')
    listTable= soup.find('div', id="tblDB")
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    limit_selector = browser.find_by_css("#pager_center select")
    #limit_selector.select(5)
    limit_selector.select(10000)
    nap()
    limit_selector.click()
    type_selector = browser.find_by_id("mainContent_ddlType2")
    type_selector.select("AL")
    nap()


In [45]:
def changeDate(date):
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    dateNow=dateField.value
    print(f'{dateNow}=>{date}')
    dateField.fill(date)
    dateSubmit.click()
    rebaseHtml()

In [17]:
def scrapeAllIncidents():
    


SyntaxError: unexpected EOF while parsing (<ipython-input-17-772796f00e11>, line 2)

In [46]:
changeDate('6/6/2015')

3/28/2021=>6/6/2015


In [47]:
changeDate('12/25/2015')

6/6/2015=>12/25/2015
